In [ ]:
import numpy as np
import pandas as pd

In [ ]:
import os

In [ ]:
from sklearn.impute import SimpleImputer

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
from sklearn.model_selection import train_test_split, KFold

In [ ]:
import matplotlib.pyplot as plt


In [ ]:
INPUT = '../input/tabular-playground-series-sep-2021'
train_path = os.path.join(INPUT, 'train.csv')
test_path = os.path.join(INPUT, 'test.csv')
train = pd.read_csv(train_path)
test = pd.read_csv(test_path)

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
train['claim'].value_counts()

In [ ]:
train.describe()

In [ ]:
test.describe()

In [ ]:
train.info()

In [ ]:
test.info()

In [ ]:
train.isnull().any().sum()

In [ ]:
test.isnull().any().sum()

In [ ]:
train.head()

In [ ]:
# missing_cols = [col for col in train.columns if train[col].isnull().any()]
feature_cols = [col for col in train.columns if 'f' in col]

In [ ]:


simple_imputer = SimpleImputer()
temp = train.drop('claim', axis=1)
imputed_train = pd.DataFrame(simple_imputer.fit_transform(temp))
imputed_test = pd.DataFrame(simple_imputer.transform(test))

imputed_train['claim'] = train['claim']

imputed_train.columns = train.columns
imputed_test.columns = test.columns



In [ ]:
imputed_train.head()

In [ ]:
type(imputed_train)

In [ ]:
imputed_train.isnull().any().sum()

In [ ]:
ss = StandardScaler()
imputed_train[feature_cols] = ss.fit_transform(imputed_train[feature_cols])
imputed_test[feature_cols] = ss.transform(imputed_test[feature_cols])

In [ ]:
y = imputed_train['claim']

In [ ]:
imputed_train.head()

In [ ]:
imputed_test.head()

In [ ]:
y.head()

In [ ]:
import optuna

In [ ]:
from xgboost import  XGBClassifier, XGBRegressor

In [ ]:
import xgboost as xgb

In [ ]:
from sklearn.metrics import mean_squared_error, accuracy_score

In [ ]:
from sklearn import metrics


In [ ]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2


In [ ]:
xgb_fi_model = XGBClassifier(tree_method='gpu_hist', use_label_encoder=False)
xgb_fi_model.fit(imputed_train[feature_cols], y)
feat_importances = pd.Series(xgb_fi_model.feature_importances_, index=feature_cols)


In [ ]:
# plt.rcParams['figure.figsize'] = [15, 18]
# feat_importances.nlargest(100).plot(kind='barh')
# plt.show()

In [ ]:
# feat_importances.keys

In [ ]:
top_feature_names = [f for f, imp in feat_importances.nlargest(100).iteritems()]
top_feature_names

In [ ]:
# top_five = top_feature_names[0:5]
# top_twenty = top_feature_names[0:20]
# next_top_thirty = top_feature_names[20:50]
# next_top_fifty = top_feature_names[50:]

In [ ]:
# def objective(trial, X=imputed_train, y=y):
# #     train_x, val_x, train_y, val_y = train_test_split(X, y, test_size=0.2,random_state=42)
#     param_space = { 
#                'lambda': trial.suggest_loguniform('lambda', 1e-3, 10.0),
#                 'alpha': trial.suggest_loguniform('alpha', 1e-3, 10.0),
#                 'colsample_bytree': trial.suggest_float('colsample_bytree', 0.1, 0.9),
#                 'colsample_bynode': trial.suggest_float('colsample_bynode', 0.1, 0.9),
#                 'colsample_bylevel': trial.suggest_float('colsample_bylevel', 0.1, 0.9),
#                 'subsample': trial.suggest_float('subsample', 0.1, 0.9),
#                 'eta':trial.suggest_float('eta', 1e-2, 1e-2),
#                 'grow_policy': trial.suggest_categorical("grow_policy", ['depthwise','lossguide']),
#                 'max_depth': trial.suggest_int('max_depth',2,25),
#                 'seed': 0,
#                 'min_child_weight': trial.suggest_int('min_child_weight', 0, 300),
#                 'max_bin': trial.suggest_int('max_bin', 256, 512),
#                 'deterministic_histogram':trial.suggest_categorical('deterministic_histogram',[False]),
#                 "tree_method" : "gpu_hist",
#                 "predictor" : 'gpu_predictor',
#                 'objective': 'binary:logistic', 

# #                  "num_class":4
#                 }
#     k=5
#     seed_list=[0]
#     kf = StratifiedKFold(n_splits=k,shuffle=True,random_state=2001)
#     oof = np.zeros(len(imputed_train))
#     score_list = []


#     splits = list(kf.split(X,y))
#     for train_idx, val_idx in splits:
#       X_train = X[X.kfold != fold].reset_index(drop=True)
#       X_val = X[X.kfold == fold].reset_index(drop=True)



#       y_train = X_train.claim
#       y_val = X_val.claim

#       X_train = X_train[top_feature_names]
#       X_val = X_val[top_feature_names]
#       val_preds_list = []

#       for seed in seed_list:
#         # fit and run model
#         param_space['seed'] = seed
#         dtrain = xgb.DMatrix(data=X_train, label=y_train)
#         dval = xgb.DMatrix(data=X_val, label=y_val)
#         dtest = xgb.DMatrix(data=imputed_test)
#         xgb.set_config(verbosity=0)


#         model = xgb.train(param_space, dtrain,\
#                        evals=[(dtrain,'train'),(dval,'val')],\
#                        verbose_eval=False,
#                        early_stopping_rounds=100,
#                        num_boost_round=100000)




#         val_preds_list.append(model.predict(dval))
#      #test_preds_list.append(model.predict_proba(imputed_test)[:,1])

#       oof[val_idx] = np.mean(val_preds_list,axis=0)
#       score = log_loss(y_val, oof[val_idx])

#       score_list.append(score)
     

#     cv_logloss = np.mean(score_list)

#     return cv_logloss

In [ ]:
from sklearn.model_selection import StratifiedKFold, KFold,cross_val_predict,cross_val_score,cross_validate

In [ ]:
# study = optuna.create_study(direction='minimize')
# study.optimize(objective, n_trials=20)
# print('Number of finished trials:', len(study.trials))
# print('Best trial:', study.best_trial.params)

In [ ]:
plt.rcParams['figure.figsize'] = [6, 6]

In [ ]:
def plot_roc(y, y_proba):
    fpr, tpr, thresholds = metrics.roc_curve(y, y_proba, pos_label = 1)
    roc_auc = metrics.auc(fpr, tpr)
    display = metrics.RocCurveDisplay(fpr=fpr, tpr=tpr, roc_auc=roc_auc)
    display.plot()
    plt.show()

In [ ]:
# xgb_params_top_twenty = {
#     'use_label_encoder': False,
#     'n_estimators': 10822,
#     'learning_rate': 0.01963896664876902,
#     'max_depth': 5,
#     'tree_method':'gpu_hist',   
#     'predictor': 'gpu_predictor',
#     'use_label_encoder':False,
#     'eval_metric':'auc'
# }

In [ ]:
# xgb_top_twenty = XGBClassifier(**xgb_params_top_twenty)

In [ ]:
# xgb_params_next_top_thirty = {
#     'use_label_encoder': False,
#     'n_estimators': 8797,
#     'learning_rate': 0.006148024937823801,
#     'max_depth': 5,
#     'tree_method':'gpu_hist',   
#     'predictor': 'gpu_predictor',
#     'use_label_encoder':False,
#     'eval_metric':'auc'
# }

In [ ]:
# xgb_next_top_thirty = XGBClassifier(**xgb_params_next_top_thirty)

In [ ]:
# xgb_params_1 = {
#     'eval_metric' : 'auc',
#     'lambda': 0.004562711234493688, 
#     'alpha': 7.268146704546314, 
#     'colsample_bytree': 0.6468987558386358, 
#     'colsample_bynode': 0.29113878257290376, 
#     'colsample_bylevel': 0.8915913499148167, 
#     'subsample': 0.37130229826185135, 
#     'learning_rate': 0.021671163563123198, 
#     'grow_policy': 'lossguide', 
#     'max_depth': 18, 
#     'min_child_weight': 215, 
#     'max_bin': 272,
#     'n_estimators': 10000,
#     'random_state': 0,
#     'use_label_encoder': False,
#     'objective': 'binary:logistic',
#     'tree_method': 'gpu_hist',
#     'gpu_id': 0,
#     'predictor': 'gpu_predictor'
# }

In [ ]:
# xgb_1 = XGBClassifier(**xgb_params_1)

In [ ]:
# xgb_params_2 = {
#     'eval_metric':'auc',
#     'max_depth':3,
#     'subsample':0.5,
#     'colsample_bytree':0.5,
#     'learning_rate':0.01187431306013263,
#     'n_estimators':10000,
#     'n_jobs':-1,
#     'use_label_encoder':False,
#     'objective':'binary:logistic',
#     'tree_method':'gpu_hist',
#     'gpu_id':0,
#     'predictor':'gpu_predictor'
# }

In [ ]:
# xgb_2 = XGBClassifier(**xgb_params_2)

In [ ]:
# xgb_params_3 = {
#     'eval_metric': 'auc', 
#     'objective': 'binary:logistic', 
#     'tree_method': 'gpu_hist', 
#     'gpu_id': 0, 
#     'predictor': 'gpu_predictor', 
#     'n_estimators': 10000, 
#     'learning_rate': 0.01063045229441343, 
#     'gamma': 0.24652519525750877, 
#     'max_depth': 4, 
#     'min_child_weight': 366, 
#     'subsample': 0.6423040816299684, 
#     'colsample_bytree': 0.7751264493218339, 
#     'colsample_bylevel': 0.8675692743597421, 
#     'lambda': 0, 
#     'alpha': 10
# }

In [ ]:
# xgb_3 = XGBClassifier(**xgb_params_3)

In [ ]:
# xgb_next_top_fifty = {
#     'use_label_encoder': False,
#     'n_estimators': 9074,
#     'learning_rate': 0.003150164455468626,
#     'max_depth': 6,
#     'tree_method':'gpu_hist',   
#     'predictor': 'gpu_predictor',
#     'use_label_encoder':False,
#     'eval_metric':'auc'
# }

In [ ]:
# xgb_next_top_fifty = XGBClassifier(**xgb_next_top_fifty)

In [ ]:
# solution_path = os.path.join(INPUT,'sample_solution.csv' )
# solution = pd.read_csv(solution_path)
# solution.head()

In [ ]:
# kf = KFold(n_splits=5, shuffle=True, random_state=42)
# for fold, (train_indices, valid_indices) in enumerate(kf.split(imputed_train)):
#     imputed_train.loc[valid_indices, 'kfold'] = fold

In [ ]:
# from catboost import CatBoostClassifier


In [ ]:
# catb_params_1 = {
#     'eval_metric' : 'AUC',
#     'iterations': 15585, 
#     'objective': 'CrossEntropy',
#     'bootstrap_type': 'Bernoulli', 
#     'od_wait': 1144, 
#     'learning_rate': 0.023575206684596582, 
#     'reg_lambda': 36.30433203563295, 
#     'random_strength': 43.75597655616195, 
#     'depth': 7, 
#     'min_data_in_leaf': 11, 
#     'leaf_estimation_iterations': 1, 
#     'subsample': 0.8227911142845009,
#     'task_type' : 'GPU',
#     'devices' : '0',
#     'verbose' : 0
# }

In [ ]:
# catb_params_2 = {
#     'eval_metric' : 'AUC',
#     'depth' : 5,
#     'grow_policy' : 'SymmetricTree',
#     'l2_leaf_reg' : 3.0,
#     'random_strength' : 1.0,
#     'learning_rate' : 0.1,
#     'iterations' : 10000,
#     'loss_function' : 'CrossEntropy',
#     'task_type' : 'GPU',
#     'devices' : '0',
#     'verbose' : 0
# }

In [ ]:
# catb_model_1 = CatBoostClassifier(**catb_params_1)

In [ ]:
# catb_model_2 = CatBoostClassifier(**catb_params_2)

In [ ]:
# solution_path = os.path.join(INPUT,'sample_solution.csv' )
# solution = pd.read_csv(solution_path)

# final_test_predictions = []
# final_valid_predictions = {}
# for fold in range(5):
#     x_train = imputed_train[imputed_train.kfold != fold].reset_index(drop=True)
#     x_val = imputed_train[imputed_train.kfold == fold].reset_index(drop=True)
    
#     valid_ids = x_val.id.values.tolist()
    
    
#     y_train = x_train.claim
#     y_val = x_val.claim
    
#     x_train = x_train[top_feature_names]
#     x_val = x_val[top_feature_names]
    
    
#     model = catb_model_1
#     model.fit(x_train, y_train)
#     preds_val = model.predict(x_val)
# #     test_preds = model.predict(test)
#     y_proba = model.predict_proba(x_val)[:,1]
#     proba = model.predict_proba(imputed_test[top_feature_names])[:,1]
#     final_test_predictions.append(proba)
#     final_valid_predictions.update(dict(zip(valid_ids, y_proba)))
#     plot_roc(y_val, y_proba)

# final_valid_predictions = pd.DataFrame.from_dict(final_valid_predictions, orient="index").reset_index()
# final_valid_predictions.columns = ["id", "pred_1"]
# final_valid_predictions.to_csv("train_pred_1.csv", index=False)

# solution.claim = np.mean(np.column_stack(final_test_predictions), axis=1)
# solution.columns = ["id", "pred_1"]
# solution.to_csv("test_pred_1.csv", index=False)

In [ ]:
# solution_path = os.path.join(INPUT,'sample_solution.csv' )
# solution = pd.read_csv(solution_path)

# final_test_predictions = []
# final_valid_predictions = {}
# for fold in range(5):
#     x_train = imputed_train[imputed_train.kfold != fold].reset_index(drop=True)
#     x_val = imputed_train[imputed_train.kfold == fold].reset_index(drop=True)
    
#     valid_ids = x_val.id.values.tolist()
    
    
#     y_train = x_train.claim
#     y_val = x_val.claim
    
#     x_train = x_train[top_feature_names]
#     x_val = x_val[top_feature_names]
    
    
#     model = catb_model_2
#     model.fit(x_train, y_train)
#     preds_val = model.predict(x_val)
# #     test_preds = model.predict(test)
#     y_proba = model.predict_proba(x_val)[:,1]
#     proba = model.predict_proba(imputed_test[top_feature_names])[:,1]
#     final_test_predictions.append(proba)
#     final_valid_predictions.update(dict(zip(valid_ids, y_proba)))
#     plot_roc(y_val, y_proba)

# final_valid_predictions = pd.DataFrame.from_dict(final_valid_predictions, orient="index").reset_index()
# final_valid_predictions.columns = ["id", "pred_6"]
# final_valid_predictions.to_csv("train_pred_6.csv", index=False)

# solution.claim = np.mean(np.column_stack(final_test_predictions), axis=1)
# solution.columns = ["id", "pred_6"]
# solution.to_csv("test_pred_6.csv", index=False)

In [ ]:
# from lightgbm import LGBMClassifier

In [ ]:
# lgbm_params_1 = {
#     'metric' : 'auc',
#     'max_depth' : 3,
#     'num_leaves' : 7,
#     'n_estimators' : 5000,
#     'colsample_bytree' : 0.3,
#     'subsample' : 0.5,
#     'random_state' : 42,
#     'reg_alpha' : 18,
#     'reg_lambda' : 17,
#     'learning_rate' : 0.095,
#     'device' : 'gpu',
#     'objective' : 'binary'
# }

In [ ]:
# lgbm_params_2 = {
#     'metric' : 'auc',
#     'objective': 'binary',
#     'n_estimators': 10000,
#     'random_state': 42,
#     'learning_rate': 0.095,
#     'subsample': 0.6,
#     'subsample_freq': 1,
#     'colsample_bytree': 0.4,
#     'reg_alpha': 10.0,
#     'reg_lambda': 1e-1,
#     'min_child_weight': 256,
#     'min_child_samples': 20,
#     'device' : 'gpu',
#     'max_depth' : 3,
#     'num_leaves' : 7
# }

In [ ]:
# lgbm_params_3 = {
#     'metric' : 'auc',
#     'objective' : 'binary',
#     'device_type': 'gpu', 
#     'n_estimators': 10000, 
#     'learning_rate': 0.12230165751633416, 
#     'num_leaves': 1400, 
#     'max_depth': 8, 
#     'min_child_samples': 3100, 
#     'reg_alpha': 10, 
#     'reg_lambda': 65, 
#     'min_split_gain': 5.157818977461183, 
#     'subsample': 0.5, 
#     'subsample_freq': 1, 
#     'colsample_bytree': 0.2
# }

In [ ]:
# lgbm_model_1 = LGBMClassifier(**lgbm_params_1)

In [ ]:
# lgbm_model_2 = LGBMClassifier(**lgbm_params_2)

In [ ]:
# lgbm_model_3 = LGBMClassifier(**lgbm_params_3)

In [ ]:
# solution_path = os.path.join(INPUT,'sample_solution.csv' )
# solution = pd.read_csv(solution_path)

# final_test_predictions = []
# final_valid_predictions = {}
# for fold in range(5):
#     x_train = imputed_train[imputed_train.kfold != fold].reset_index(drop=True)
#     x_val = imputed_train[imputed_train.kfold == fold].reset_index(drop=True)
    
#     valid_ids = x_val.id.values.tolist()
    
    
#     y_train = x_train.claim
#     y_val = x_val.claim
    
#     x_train = x_train[top_feature_names]
#     x_val = x_val[top_feature_names]
    
    
#     model = lgbm_model_1
#     model.fit(x_train, y_train)
#     preds_val = model.predict(x_val)
# #     test_preds = model.predict(test)
#     y_proba = model.predict_proba(x_val)[:,1]
#     proba = model.predict_proba(imputed_test[top_feature_names])[:,1]
#     final_test_predictions.append(proba)
#     final_valid_predictions.update(dict(zip(valid_ids, y_proba)))
#     plot_roc(y_val, y_proba)

# final_valid_predictions = pd.DataFrame.from_dict(final_valid_predictions, orient="index").reset_index()
# final_valid_predictions.columns = ["id", "pred_2"]
# final_valid_predictions.to_csv("train_pred_2.csv", index=False)

# solution.claim = np.mean(np.column_stack(final_test_predictions), axis=1)
# solution.columns = ["id", "pred_2"]
# solution.to_csv("test_pred_2.csv", index=False)

In [ ]:
# solution_path = os.path.join(INPUT,'sample_solution.csv' )
# solution = pd.read_csv(solution_path)

# final_test_predictions = []
# final_valid_predictions = {}
# for fold in range(5):
#     x_train = imputed_train[imputed_train.kfold != fold].reset_index(drop=True)
#     x_val = imputed_train[imputed_train.kfold == fold].reset_index(drop=True)
    
#     valid_ids = x_val.id.values.tolist()
    
    
#     y_train = x_train.claim
#     y_val = x_val.claim
    
#     x_train = x_train[top_feature_names]
#     x_val = x_val[top_feature_names]
    
    
#     model = lgbm_model_2
#     model.fit(x_train, y_train)
#     preds_val = model.predict(x_val)
# #     test_preds = model.predict(test)
#     y_proba = model.predict_proba(x_val)[:,1]
#     proba = model.predict_proba(imputed_test[top_feature_names])[:,1]
#     final_test_predictions.append(proba)
#     final_valid_predictions.update(dict(zip(valid_ids, y_proba)))
#     plot_roc(y_val, y_proba)

# final_valid_predictions = pd.DataFrame.from_dict(final_valid_predictions, orient="index").reset_index()
# final_valid_predictions.columns = ["id", "pred_5"]
# final_valid_predictions.to_csv("train_pred_5.csv", index=False)

# solution.claim = np.mean(np.column_stack(final_test_predictions), axis=1)
# solution.columns = ["id", "pred_5"]
# solution.to_csv("test_pred_5.csv", index=False)

In [ ]:
# solution_path = os.path.join(INPUT,'sample_solution.csv' )
# solution = pd.read_csv(solution_path)

# final_test_predictions = []
# final_valid_predictions = {}
# for fold in range(5):
#     x_train = imputed_train[imputed_train.kfold != fold].reset_index(drop=True)
#     x_val = imputed_train[imputed_train.kfold == fold].reset_index(drop=True)
    
#     valid_ids = x_val.id.values.tolist()
    
    
#     y_train = x_train.claim
#     y_val = x_val.claim
    
#     x_train = x_train[top_feature_names]
#     x_val = x_val[top_feature_names]
    
    
#     model = lgbm_model_3
#     model.fit(x_train, y_train)
#     preds_val = model.predict(x_val)
# #     test_preds = model.predict(test)
#     y_proba = model.predict_proba(x_val)[:,1]
#     proba = model.predict_proba(imputed_test[top_feature_names])[:,1]
#     final_test_predictions.append(proba)
#     final_valid_predictions.update(dict(zip(valid_ids, y_proba)))
#     plot_roc(y_val, y_proba)

# final_valid_predictions = pd.DataFrame.from_dict(final_valid_predictions, orient="index").reset_index()
# final_valid_predictions.columns = ["id", "pred_8"]
# final_valid_predictions.to_csv("train_pred_8.csv", index=False)

# solution.claim = np.mean(np.column_stack(final_test_predictions), axis=1)
# solution.columns = ["id", "pred_8"]
# solution.to_csv("test_pred_8.csv", index=False)

In [ ]:
# solution_path = os.path.join(INPUT,'sample_solution.csv' )
# solution = pd.read_csv(solution_path)

# final_test_predictions = []
# final_valid_predictions = {}
# for fold in range(5):
#     x_train = imputed_train[imputed_train.kfold != fold].reset_index(drop=True)
#     x_val = imputed_train[imputed_train.kfold == fold].reset_index(drop=True)
    
#     valid_ids = x_val.id.values.tolist()
    
    
#     y_train = x_train.claim
#     y_val = x_val.claim
    
#     x_train = x_train[top_feature_names]
#     x_val = x_val[top_feature_names]
    
    
#     model = xgb_1
#     model.fit(x_train, y_train)
#     preds_val = model.predict(x_val)
# #     test_preds = model.predict(test)
#     y_proba = model.predict_proba(x_val)[:,1]
#     proba = model.predict_proba(imputed_test[top_feature_names])[:,1]
#     final_test_predictions.append(proba)
#     final_valid_predictions.update(dict(zip(valid_ids, y_proba)))
#     plot_roc(y_val, y_proba)

# final_valid_predictions = pd.DataFrame.from_dict(final_valid_predictions, orient="index").reset_index()
# final_valid_predictions.columns = ["id", "pred_3"]
# final_valid_predictions.to_csv("train_pred_3.csv", index=False)

# solution.claim = np.mean(np.column_stack(final_test_predictions), axis=1)
# solution.columns = ["id", "pred_3"]
# solution.to_csv("test_pred_3.csv", index=False)

In [ ]:
# solution_path = os.path.join(INPUT,'sample_solution.csv' )
# solution = pd.read_csv(solution_path)

# final_test_predictions = []
# final_valid_predictions = {}
# for fold in range(5):
#     x_train = imputed_train[imputed_train.kfold != fold].reset_index(drop=True)
#     x_val = imputed_train[imputed_train.kfold == fold].reset_index(drop=True)
    
#     valid_ids = x_val.id.values.tolist()
    
    
#     y_train = x_train.claim
#     y_val = x_val.claim
    
#     x_train = x_train[top_feature_names]
#     x_val = x_val[top_feature_names]
    
    
#     model = xgb_2
#     model.fit(x_train, y_train)
#     preds_val = model.predict(x_val)
# #     test_preds = model.predict(test)
#     y_proba = model.predict_proba(x_val)[:,1]
#     proba = model.predict_proba(imputed_test[top_feature_names])[:,1]
#     final_test_predictions.append(proba)
#     final_valid_predictions.update(dict(zip(valid_ids, y_proba)))
#     plot_roc(y_val, y_proba)

# final_valid_predictions = pd.DataFrame.from_dict(final_valid_predictions, orient="index").reset_index()
# final_valid_predictions.columns = ["id", "pred_4"]
# final_valid_predictions.to_csv("train_pred_4.csv", index=False)

# solution.claim = np.mean(np.column_stack(final_test_predictions), axis=1)
# solution.columns = ["id", "pred_4"]
# solution.to_csv("test_pred_4.csv", index=False)

In [ ]:
# solution_path = os.path.join(INPUT,'sample_solution.csv' )
# solution = pd.read_csv(solution_path)

# final_test_predictions = []
# final_valid_predictions = {}
# for fold in range(5):
#     x_train = imputed_train[imputed_train.kfold != fold].reset_index(drop=True)
#     x_val = imputed_train[imputed_train.kfold == fold].reset_index(drop=True)
    
#     valid_ids = x_val.id.values.tolist()
    
    
#     y_train = x_train.claim
#     y_val = x_val.claim
    
#     x_train = x_train[top_feature_names]
#     x_val = x_val[top_feature_names]
    
    
#     model = xgb_3
#     model.fit(x_train, y_train)
#     preds_val = model.predict(x_val)
# #     test_preds = model.predict(test)
#     y_proba = model.predict_proba(x_val)[:,1]
#     proba = model.predict_proba(imputed_test[top_feature_names])[:,1]
#     final_test_predictions.append(proba)
#     final_valid_predictions.update(dict(zip(valid_ids, y_proba)))
#     plot_roc(y_val, y_proba)

# final_valid_predictions = pd.DataFrame.from_dict(final_valid_predictions, orient="index").reset_index()
# final_valid_predictions.columns = ["id", "pred_7"]
# final_valid_predictions.to_csv("train_pred_7.csv", index=False)

# solution.claim = np.mean(np.column_stack(final_test_predictions), axis=1)
# solution.columns = ["id", "pred_7"]
# solution.to_csv("test_pred_7.csv", index=False)

In [ ]:
# solution_path = os.path.join(INPUT,'sample_solution.csv' )
# solution = pd.read_csv(solution_path)

# final_test_predictions = []
# final_valid_predictions = {}
# for fold in range(5):
#     x_train = imputed_train[imputed_train.kfold != fold].reset_index(drop=True)
#     x_val = imputed_train[imputed_train.kfold == fold].reset_index(drop=True)
    
#     valid_ids = x_val.id.values.tolist()
    
    
#     y_train = x_train.claim
#     y_val = x_val.claim
    
#     x_train = x_train[top_feature_names]
#     x_val = x_val[top_feature_names]
    
    
#     model = xgb
#     model.fit(x_train, y_train, early_stopping_rounds=100, eval_set=[[x_val, y_val]], verbose=500)
#     preds_val = model.predict(x_val)
# #     test_preds = model.predict(test)
#     y_proba = model.predict_proba(x_val)[:,1]
#     proba = model.predict_proba(imputed_test[top_feature_names])[:,1]
#     final_test_predictions.append(proba)
#     final_valid_predictions.update(dict(zip(valid_ids, y_proba)))
#     plot_roc(y_val, y_proba)

# final_valid_predictions = pd.DataFrame.from_dict(final_valid_predictions, orient="index").reset_index()
# final_valid_predictions.columns = ["id", "pred_4"]
# final_valid_predictions.to_csv("train_pred_4.csv", index=False)

# solution.claim = np.mean(np.column_stack(final_test_predictions), axis=1)
# solution.columns = ["id", "pred_4"]
# solution.to_csv("test_pred_4.csv", index=False)

In [ ]:
# xgb_predictions = []
# for fold in range(10):
#     x_train = imputed_train[imputed_train.kfold != fold].reset_index(drop=True)
#     x_val = imputed_train[imputed_train.kfold == fold].reset_index(drop=True)
    
#     y_train = x_train.claim
#     y_val = x_val.claim
    
#     x_train = x_train[feature_cols]
#     x_val = x_val[feature_cols]
    
#     model = xgb
#     model.fit(x_train, y_train, early_stopping_rounds=100, eval_set=[[x_val, y_val]], verbose=500)
#     preds_val = model.predict(x_val)
# #     test_preds = model.predict(test)
#     y_proba = model.predict_proba(x_val)[:,1]
#     proba = model.predict_proba(imputed_test[feature_cols])[:,1]
#     xgb_predictions.append(proba)
#     plot_roc(y_val, y_proba)
    

In [ ]:
# solution_path = os.path.join(INPUT,'sample_solution.csv' )
# solution = pd.read_csv(solution_path)

# df1 = pd.read_csv("train_pred_1.csv")
# df2 = pd.read_csv("train_pred_2.csv")
# df3 = pd.read_csv("train_pred_3.csv")
# df4 = pd.read_csv("train_pred_4.csv")
# df5 = pd.read_csv("train_pred_5.csv")
# df6 = pd.read_csv("train_pred_6.csv")
# df7 = pd.read_csv("train_pred_7.csv")
# df8 = pd.read_csv("train_pred_8.csv")

# df_test1 = pd.read_csv("test_pred_1.csv")
# df_test2 = pd.read_csv("test_pred_2.csv")
# df_test3 = pd.read_csv("test_pred_3.csv")
# df_test4 = pd.read_csv("test_pred_4.csv")
# df_test5 = pd.read_csv("test_pred_5.csv")
# df_test6 = pd.read_csv("test_pred_6.csv")
# df_test7 = pd.read_csv("test_pred_7.csv")
# df_test8 = pd.read_csv("test_pred_8.csv")




# imputed_train = imputed_train.merge(df1, on="id", how="left")
# imputed_train = imputed_train.merge(df2, on="id", how="left")
# imputed_train = imputed_train.merge(df3, on="id", how="left")
# imputed_train = imputed_train.merge(df4, on="id", how="left")
# imputed_train = imputed_train.merge(df5, on="id", how="left")
# imputed_train = imputed_train.merge(df6, on="id", how="left")
# imputed_train = imputed_train.merge(df7, on="id", how="left")
# imputed_train = imputed_train.merge(df8, on="id", how="left")




# imputed_test = imputed_test.merge(df_test1, on="id", how="left")
# imputed_test = imputed_test.merge(df_test2, on="id", how="left")
# imputed_test = imputed_test.merge(df_test3, on="id", how="left")
# imputed_test = imputed_test.merge(df_test4, on="id", how="left")
# imputed_test = imputed_test.merge(df_test5, on="id", how="left")
# imputed_test = imputed_test.merge(df_test6, on="id", how="left")
# imputed_test = imputed_test.merge(df_test7, on="id", how="left")
# imputed_test = imputed_test.merge(df_test8, on="id", how="left")



# imputed_train.head()


In [ ]:
# imputed_test.head()

In [ ]:
# final_predictions = xgb_predictions

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
useful_features = ["pred_1", "pred_2", "pred_3", "pred_4", "pred_5", "pred_6", "pred_7", "pred_8"]

final_predictions = []
scores = []
for fold in range(5):
    xtrain =  imputed_train[imputed_train.kfold != fold].reset_index(drop=True)
    xvalid = imputed_train[imputed_train.kfold == fold].reset_index(drop=True)
    xtest = imputed_test.copy()

    ytrain = xtrain.claim
    yvalid = xvalid.claim
    
    xtrain = xtrain[useful_features]
    xvalid = xvalid[useful_features]
    
    model = XGBRegressor(
            tree_method='gpu_hist',   
            predictor='gpu_predictor'
        )

#     model = LinearRegression()
    model.fit(xtrain, ytrain)
    
    preds_valid = model.predict(xvalid)
    test_preds = model.predict(xtest[useful_features])
    final_predictions.append(test_preds)
    rmse = mean_squared_error(yvalid, preds_valid, squared=False)
    print(fold, rmse)
    scores.append(rmse)

print(np.mean(scores), np.std(scores))

In [ ]:

predictions = np.mean(np.column_stack(final_predictions), axis=1)


In [ ]:
solution['claim'] = predictions
# submission['loss'] = final_predictions
solution.to_csv("solution.csv", index=False)

solution